In [1]:
import os
import cv2
from tqdm import tqdm
import math
from pathlib import Path
import numpy as np

In [2]:
BASE_PATH = '/home/basidio/Development/omscs/dl/DiamondsInTheRough/minerl2020_pdddqn_submission/modified_arch_never_frozen/85a45807b34265153c1859459809b9d49691a674-ffc71b04-3635a7a8/MineRLTreechopVectorObf-v0/monitor/'
EPOCHS    = [0, 50, 100, 150, 200, 250, 300, 329]
OUT_PATH  = '/home/basidio/Development/omscs/dl/DiamondsInTheRough/youtube_famous/videos/with_pretrained_cae.mp4'

In [3]:
def get_mp4_path(epoch):
    global BASE_PATH
    filename = f"openaigym.video.0.2872011.video{epoch:06d}.mp4"
    return os.path.join(BASE_PATH, filename)

video_names_and_paths = [(f'Epoch {epoch}', get_mp4_path(epoch)) for epoch in EPOCHS]
assert len(video_names_and_paths) > 0

In [4]:
def get_layout(num_videos):
    height  = int(math.sqrt(num_videos))
    width = int(math.ceil(num_videos / height))
    return height, width

video_layout = get_layout(len(EPOCHS))

In [5]:
def get_video_shape_fps_framecount_dtype(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    ret, frame = cap.read()
    cap.release()
    assert ret == True
    return frame.shape, fps, frame_count, frame.dtype

def get_out_shape(layout, subvideo_shape):
    return subvideo_shape[0] * layout[0], subvideo_shape[1] * layout[1], subvideo_shape[2]

def get_out_framecount(video_paths):
    assert len(video_paths) > 0

    max_fc = 0
    for video_path in video_paths:
        assert os.path.exists(video_path)
        _, _, fc, _  = get_video_shape_fps_framecount_dtype(video_path)
        if fc > max_fc:
            max_fc = fc
    assert max_fc > 0
    return max_fc
        

subvideo_shape, out_fps, _, out_dtype = get_video_shape_fps_framecount_dtype(video_names_and_paths[0][1])
out_shape                             = get_out_shape(video_layout, subvideo_shape)
out_framecount                        = get_out_framecount(list(zip(*video_names_and_paths))[1])

In [16]:
def generate_frame(sub_frames, video_layout):
    assert len(sub_frames) > 0
    subframe_h, subframe_w, _ = sub_frames[0].shape
    frame_dtype               = sub_frames[0].dtype
    frame_shape = get_out_shape(video_layout, sub_frames[0].shape)
    frame = np.zeros(frame_shape, frame_dtype)
    
    row_i, col_i = 0, 0
    for sub_frame in sub_frames:
        start_h = row_i * subframe_h
        end_h   = start_h + subframe_h
        start_w = col_i * subframe_w
        end_w   = start_w + subframe_w
        
        frame[start_h:end_h, start_w:end_w, :] = sub_frame
        
        col_i = col_i + 1
        if col_i == video_layout[1]:
            col_i = 0
            row_i += 1

    return frame   

def write_text(label, subframe):
    font = cv2.FONT_HERSHEY_SIMPLEX 

    org       = (1, 10) 
    fontScale = 0.2
    color     = (255, 0, 255) # Purple
    thickness = 1

    subframe_with_text = cv2.putText(subframe, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
    return subframe_with_text

def generate_subframe(label, cap, subvideo_shape, out_dtype):
    ret, frame = cap.read()
    out_frame  = frame if ret else np.zeros(subvideo_shape, dtype=out_dtype)
    out_frame  = write_text(label, out_frame)
    return out_frame

In [17]:
containing_dir = Path(OUT_PATH).parent
os.makedirs(containing_dir, exist_ok=True)
if os.path.exists(OUT_PATH):
    os.remove(OUT_PATH)
out_writer = cv2.VideoWriter(OUT_PATH, cv2.VideoWriter_fourcc(*'MP4V'), out_fps, (out_shape[1], out_shape[0]))

print(f'Writing frames to {OUT_PATH}')

video_names_and_caps = [(name, cv2.VideoCapture(mp4_path)) for name, mp4_path in video_names_and_paths]
for _ in tqdm(range(0, out_framecount)):
    subframes = np.array([generate_subframe(label, cap, subvideo_shape, out_dtype) for label, cap in video_names_and_caps])
    frame     = generate_frame(subframes, video_layout)
    out_writer.write(frame)

out_writer.release()
for _, cap in video_names_and_caps:
    cap.release()

  0%|          | 1/8001 [00:00<13:33,  9.84it/s]

Writing frames to /home/basidio/Development/omscs/dl/DiamondsInTheRough/youtube_famous/videos/with_pretrained_cae.mp4


100%|██████████| 8001/8001 [00:07<00:00, 1067.76it/s]
